In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']=(5.0,4.0) #set default size of plot
plt.rcParams['image.interpolation']='nearest'
plt.rcParams['image.cmap']='gray'
np.random.seed(1)

In [3]:
def sigmoid(Z):
    A=1/(1+np.exp(-Z)) #sigmoid eqn
    cache=Z
    
    return A,cache

def relu(Z):
    A=np.maximum(0,Z)
    cache=Z
    return A,cache

def relu_backward(dA,cache):
    Z=cache
    dZ=np.array(dA,copy=True) #dA -differntiate of A
    dZ[Z<=0]=0
    return dZ

def sigmoid_backward(dA,cache):
    Z=cache
    s=1/(1+np.exp(-z))
    dZ=dA*s*(1-s) #differntiate of sigmoid
    
    return dZ

##### Steps for building L layer of Neural Network in Deep Learning

- Initialize parameter  (W1,b1,W2,b2)
- Forward propogation
- Find cost fun
- Backward propogation

#### 1 .Initialization

- The model's Structure is: LINEAR->RELU->LINEAR->SIGMOID
- use this random initialization for the weight matrices:np.random.randn(shape)*0.01 with the correct shape
- use Zero initialization for the biases:np.zeros(shape)


In [8]:
def initialize_parameter(n_x,n_h,n_y):
    w1=np.random.randn(n_h,n_x)*0.01
    b1=np.zeros([n_h,1])
    w2=np.random.randn(n_y,n_h)*0.01
    b2=np.zeros([n_y,1])
    
    parameter={'w1':w1,
              'b1':b1,
              'w2':w2,
              'b2':b2}
    
    return parameter
initialize_parameter(4,2,3)

{'w1': array([[-0.00267888,  0.00530355, -0.00691661, -0.00396754],
        [-0.00687173, -0.00845206, -0.00671246, -0.00012665]]),
 'b1': array([[0.],
        [0.]]),
 'w2': array([[-0.0111731 ,  0.00234416],
        [ 0.01659802,  0.00742044],
        [-0.00191836, -0.00887629]]),
 'b2': array([[0.],
        [0.],
        [0.]])}